In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import urllib.request
import urllib.error
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import re
%matplotlib inline

In [2]:
#Dictionary of team names and abbreviations. VEG is used instead of VGK.
teamAbvs = {'Anaheim Ducks':'ANA', 'Arizona Coyotes':'ARI', 'Boston Bruins': 'BOS', 'Buffalo Sabres':'BUF', 'Carolina Hurricanes':'CAR',
            'Calgary Flames':'CGY', 'Chicago Blackhawks':'CHI', 'Columbus Blue Jackets':'CBJ', 'Colorado Avalanche':'COL', 'Dallas Stars':'DAL',
           'Detroit Red Wings':'DET', 'Edmonton Oilers':'EDM', 'Florida Panthers':'FLA', 'Los Angeles Kings':'LAK',
           'Minnesota Wild':'MIN', 'Montreal Canadiens':'MTL', 'Nashville Predators':'NSH', 'New Jersey Devils':'NJD',
           'New York Islanders':'NYI', 'New York Rangers':'NYR', 'Ottawa Senators':'OTT', 'Phoenix Coyotes':'PHX', 'Philadelphia Flyers':'PHI',
           'Pittsburgh Penguins':'PIT', 'San Jose Sharks':'SJS', 'St. Louis Blues':'STL', 'Tampa Bay Lightning':'TBL',
           'Toronto Maple Leafs':'TOR','Vancouver Canucks':'VAN', 'Vegas Golden Knights':'VEG', 'Washington Capitals':'WSH', 'Winnipeg Jets':'WPG',
           }

In [76]:
teams = sorted(list(teamAbvs.values()))
penalties = ['Slashing', 'Hooking', 'Tripping', 'Highsticking', 'Interference', 'Roughing', 'Fighting', 'Total']
games = np.arange(1,1231,1)
team_penalties = pd.DataFrame(np.zeros((1230,256), dtype='int32'),
                              index=games,
                              columns=pd.MultiIndex.from_product([teams, penalties], names=['Team', 'Penalty'])
                             )
team_penalties.index.name='Game'
team_penalties.to_csv('test_teams', index=False)

In [83]:
rt = ['blah', 'bleh', 'BOS']
name = [word for word in rt if word.upper() in teams][0]
team_penalties[name]

Penalty,Slashing,Hooking,Tripping,Highsticking,Interference,Roughing,Fighting,Total
Game,,,,,,,,
1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0


In [66]:
def startSeason(season_url):
    season_year = season_url.split('_')[1]
    
    #Retrieving the initial url and HTML
    season_page = urlopen(season_url)
    season_soup = BeautifulSoup(season_page)
    
    #Building the schedule table
    season_games = season_soup.find(id='games')
    Date =[]
    Vis=[]
    VG=[]
    Home=[]
    HG=[]
    OT=[]
    Att=[]
    LOG=[]
    Notes=[]

    #Create a table with a list for each column of data
    headers = [Date, Vis, VG, Home, HG, OT, Att, LOG, Notes]
    
    #Iterate through each row and column, assigning the corresponding data to the location in 'headers'
    for row in season_games.findAll('tr'):
        cells = row.findAll(['td','th'])
        index = np.arange(0,len(cells)).tolist()
        for num in index:
            headers[num].append(cells[num].find(text=True))
            
    #Create the games Dataframe, set the column names to names instead of index values
    season_gamesDf = pd.DataFrame(headers).transpose()
    season_gamesDf.columns = season_gamesDf.iloc[0]
    season_gamesDf = season_gamesDf.reindex(season_gamesDf.index.drop(0))
    
    #Create game HTML string for every game
    gameHTML = []
    for entry in season_gamesDf.index:
        abv = teamAbvs[season_gamesDf['Home'][entry]]
        daystr = ''.join(season_gamesDf['Date'][entry].split('-'))
        gamestr = (daystr+'0'+abv)
        gamepage = ('https://www.hockey-reference.com/boxscores/'+gamestr+'.html')
        gameHTML.append(gamepage)
    #Add the new gameHTML column to gamesDf
    season_gamesDf['Game HTML'] = gameHTML
    number_of_games = len(gameHTML)

    #Create the penalty columns for each game and the columns for running totals of each penalty
    initialCalls = np.zeros(number_of_games)
    initialCalls.tolist()
    season_gamesDf['Slashing Calls in Game'] = initialCalls 
    season_gamesDf['Total Slashing Calls'] = initialCalls  

    season_gamesDf['roughing Calls in Game'] = initialCalls 
    season_gamesDf['Total roughing Calls'] = initialCalls 

    season_gamesDf['hooking Calls in Game'] = initialCalls  
    season_gamesDf['Total hooking Calls'] = initialCalls  

    season_gamesDf['interfering Calls in Game'] = initialCalls  
    season_gamesDf['Total interfering Calls'] = initialCalls  

    season_gamesDf['tripping Calls in Game'] = initialCalls  
    season_gamesDf['Total tripping Calls'] = initialCalls  

    season_gamesDf['highsticking Calls in Game'] = initialCalls  
    season_gamesDf['Total highsticking Calls'] = initialCalls 

    season_gamesDf['fighting Calls in Game'] = initialCalls 
    season_gamesDf['Total fighting Calls'] = initialCalls 
    
    #Get all the column names for the overall penalty table
    penalty_columns = season_gamesDf.columns.tolist()

    #Output the dataframe object with all of the retrieved data
    season_gamesDf.to_csv('games'+season_year+'dataframe_allpenalties', index=False,  header=penalty_columns)
    
    #Creating an empty dataframe for penalties for each team in the season
    teams = sorted(list(teamAbvs.values()))
    penalties = ['Slashing', 'Hooking', 'Tripping', 'Highsticking', 'Interference', 'Roughing', 'Fighting', 'Total']
    games = np.arange(1,number_of_games+1,1)
    team_penalties = pd.DataFrame(np.zeros((number_of_games,256), dtype='int32'),
                                  index=games,
                                  columns=pd.MultiIndex.from_product([teams, penalties], names=['Team', 'Penalty'])
                                 )
    
    team_penalties.to_csv('team_penalties'+season_year, index=False)

In [67]:
startSeason('https://www.hockey-reference.com/leagues/NHL_2019_games.html')

In [5]:
def updateSeason(season_url):
    #Need to check that the season has already been 'started' by startSeason() and the dfs exist.
    import os.path
    season_year = season_url.split('_')[1]
    
    season_df_name = 'games'+season_year+'dataframe_allpenalties'
    team_season_df = 'team_penalties'+season_year
    
    if os.path.isfile(season_df_name) == False:
        return 'Season data does not exist, has the season been initialized with startSeason?'
    else:
        local_season_df = pd.read_csv(season_df_name)
    if os.path.isfile(team_season_df) == False:
        return 'Teams data does not exist, has the season been initialized'
    else:
        local_team_season_df = pd.read_csv(team_season_df, header=[0,1])
    
    #Find the last row with valid data and pick the next one as the next row to be entered.
    #Used LOG column because without a valid LOG it should be safe to assume there was no game.
    next_entry = local_season_df['LOG'].last_valid_index()+1
    columns = local_season_df.columns.tolist()
    
    #Need to rebuild the existing online table to compare to local data
    #Retrieving the initial url and HTML
    season_page = urlopen(season_url)
    season_soup = BeautifulSoup(season_page)
    
    #Building the schedule table
    season_games = season_soup.find(id='games')
    Date =[]
    Vis=[]
    VG=[]
    Home=[]
    HG=[]
    OT=[]
    Att=[]
    LOG=[]
    Notes=[]

    #Create a table with a list for each column of data
    headers = [Date, Vis, VG, Home, HG, OT, Att, LOG, Notes]
    
    #Iterate through each row and column, assigning the corresponding data to the location in 'headers'
    for row in season_games.findAll('tr'):
        cells = row.findAll(['td','th'])
        index = np.arange(0,len(cells)).tolist()
        for num in index:
            headers[num].append(cells[num].find(text=True))
            
    #Create the games Dataframe, set the column names to names instead of index values
    season_gamesDf = pd.DataFrame(headers).transpose()
    season_gamesDf.columns = season_gamesDf.iloc[0]
    season_gamesDf = season_gamesDf.reindex(season_gamesDf.index.drop(0))
    
    #Find the most recent game updated. Defines the span of games which need to be updated.
    latest_entry = season_gamesDf['LOG'].last_valid_index()
    
    while next_entry < latest_entry:
        gamepage = local_season_df['Game HTML'][next_entry]
        openedgame = urlopen(gamepage)
        gamesoup = BeautifulSoup(openedgame)    
        penaltytable = gamesoup.find(id='penalty')
        slashingInGame = 0
        roughingInGame = 0
        trippingInGame = 0
        interferenceInGame = 0
        hookingInGame = 0
        highstickingInGame = 0
        fightingInGame = 0
        #Going through the penalty entries to find slashing calls
        for row in penaltytable.findAll('td'):
            rowtext = row.get_text().lower().split()
            if 'slashing' in rowtext:
                slashingInGame += 1.0
            if 'fighting' in rowtext:
                fightingInGame += 1.0
            if 'hooking' in rowtext:
                hookingInGame += 1.0
            if 'tripping' in rowtext:
                trippingInGame += 1.0
            if 'roughing' in rowtext:
                roughingInGame += 1.0
            if 'sticking' in rowtext:
                highstickingInGame += 1.0
            if 'interference' in rowtext:
                interferenceInGame += 1.0
        #Adding relevant entries to each list
        #Using next_entry values to avoid indexing issues
        local_season_df.iloc[next_entry, 10] = slashingInGame
        local_season_df.iloc[next_entry, 12] = roughingInGame
        local_season_df.iloc[next_entry, 14] = hookingInGame
        local_season_df.iloc[next_entry, 16] = interferenceInGame
        local_season_df.iloc[next_entry, 18] = trippingInGame
        local_season_df.iloc[next_entry, 20] = highstickingInGame
        local_season_df.iloc[next_entry, 22] = fightingInGame
        if next_entry == 1:
            local_season_df.iloc[next_entry, 11] = slashingInGame
            local_season_df.iloc[next_entry, 13] = roughingInGame
            local_season_df.iloc[next_entry, 15] = hookingInGame
            local_season_df.iloc[next_entry, 17] = interferenceInGame
            local_season_df.iloc[next_entry, 19] = trippingInGame
            local_season_df.iloc[next_entry, 21] = highstickingInGame
            local_season_df.iloc[next_entry, 23] = fightingInGame
        else:
            slashupdatedtotal = (local_season_df['Total Slashing Calls'][(next_entry-1)] + slashingInGame)
            local_season_df.iloc[next_entry, 11] = slashupdatedtotal
            roughupdatedtotal = (local_season_df['Total roughing Calls'][(next_entry-1)] + roughingInGame)
            local_season_df.iloc[next_entry, 13] = roughupdatedtotal
            hookupdatedtotal = (local_season_df['Total hooking Calls'][(next_entry-1)] + hookingInGame)
            local_season_df.iloc[next_entry, 15] = hookupdatedtotal
            interupdatedtotal = (local_season_df['Total interfering Calls'][(next_entry-1)] + interferenceInGame)
            local_season_df.iloc[next_entry, 17] = interupdatedtotal
            tripupdatedtotal = (local_season_df['Total tripping Calls'][(next_entry-1)] + trippingInGame)
            local_season_df.iloc[next_entry, 19] = tripupdatedtotal
            stickupdatedtotal = (local_season_df['Total highsticking Calls'][(next_entry-1)] + highstickingInGame)
            local_season_df.iloc[next_entry, 21] = stickupdatedtotal
            fightupdatedtotal = (local_season_df['Total fighting Calls'][(next_entry-1)] + fightingInGame)
            local_season_df.iloc[next_entry, 23] = fightupdatedtotal
        print(local_season_df['Total Slashing Calls'][next_entry])
        next_entry += 1
    
    #Save the updated data and overwrite the previous file
    local_season_df.to_csv('games'+season_year+'dataframe_allpenalties', index = False,  header=columns)

In [6]:
updateSeason('https://www.hockey-reference.com/leagues/NHL_2017_games.html')

In [ ]:
##Want to update with: handling of cancelled games, recording of team penalty totals##